In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import avg, count, countDistinct, max, min, mean, variance, stddev, sum, skewness, kurtosis
from pyspark.sql.functions import hour,minute,second,year,month,dayofmonth,date_format
from pyspark.sql.functions import col, lit
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from time import sleep
import csv
from fitter import Fitter
import os
import shutil
import gc

In [2]:
spark = SparkSession.builder.\
    appName("local[*]").\
    getOrCreate()

22/04/15 22:10:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
st_struct = {}
all_struct = {}

In [9]:
%%time
@udf(returnType=StringType())
def add_(a):
    return str(a)+'$_$'
for i in tqdm(range(6, 10)):
    #遍历每一个月
    path = "../data_20210{}.tsv".format(i)
    df = spark.read.option("encoding","gb18030").csv(path,sep='\t',header=True)
    
    #预处理
    df =  df.filter(df['ITEM_SALES_VOLUME'] != 'null')
    df = df.na.fill('Unkown')
    df = df.withColumn('CATE_NAME_LV', F.concat(df['CATE_NAME_LV1'], df['CATE_NAME_LV2'], df['CATE_NAME_LV3'], df['CATE_NAME_LV4'],df['CATE_NAME_LV5']))
    
    #建表，保存数据，再一行一行处理
    df.createOrReplaceTempView("data")
    if not os.path.exists('./P_{}'.format(i)):
        tmp = spark.sql('select DATA_MONTH, ITEM_ID, ITEM_SALES_VOLUME, CATE_NAME_LV, ITEM_PRICE from data')
        tmp.repartition(1).write.option("encoding","gb18030").csv('./P_{}'.format(i))
    for file in os.listdir("./P_{}".format(i)):
        if file.endswith(".csv"):
            with open("./P_{}/{}".format(i, file), encoding='gb18030') as f:
                reader = csv.reader(f)
                for tmp in tqdm(reader):
                    if tmp[1] in st_struct:
                        st_struct[tmp[1]][tmp[0]] = int(tmp[2])
                    else:
                        st_struct[tmp[1]] = {}
                        st_struct[tmp[1]][tmp[0]] = int(tmp[2])
    del(df)
    gc.collect()

  0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

CPU times: user 55.6 s, sys: 4.28 s, total: 59.9 s
Wall time: 2min 17s


# 保存id和月份信息

In [10]:
st_1 = {}
ans = {}

In [11]:
xx = 0
f = open('./falg1.csv', 'w', encoding='gb18030', newline='')
writer = csv.writer(f)
ff = open('./falg-1.csv', 'w', encoding='gb18030', newline='')
writerf = csv.writer(ff)
for i in tqdm(st_struct):
    if(len(st_struct[i]) == 1):
        st_1[i] = st_struct[i]
        x = [i] + list(st_struct[i].keys())
        writerf.writerow(x)
    else:
        try:
            sum, tmp = np.sum(list(st_struct[i].values())), 1
        except:
            print(st_struct[i].values())
        for j in st_struct[i]:
            try:
                avg = (sum - st_struct[i][j]) / (len(st_struct[i])-1)
            except:
                print(st_struct[i].values())
            if 0 <= avg < 10:
                tmp = 100
            elif 10 <= avg < 100:
                tmp = 50
            elif 100 <= avg < 1000:
                tmp = 10
            elif 1000 <= avg < 100000:
                tmp = 7
            elif 10000 <= avg < 100000:
                tmp = 5
            else:
                tmp = 2
            if st_struct[i][j] > tmp * avg:
                xx += 1
                if i not in ans:
                    ans[i] = {}
                ans[str(i)][str(j)] = 1
                writer.writerow([i, j])
f.close()
ff.close()

  0%|          | 0/7859140 [00:00<?, ?it/s]

In [12]:
st_1 = {}
ans = {}
with open("./falg-1.csv", encoding='gb18030') as f:
    reader = csv.reader(f)
    for tmp in tqdm(reader):
        st_1[tmp[0]+'$_$'+tmp[1]] = 1

with open("./falg1.csv", encoding='gb18030') as f:
    reader = csv.reader(f)
    for tmp in tqdm(reader):
        ans[tmp[0]+'$_$'+tmp[1]] = 1

0it [00:00, ?it/s]

0it [00:00, ?it/s]

# 给原数据打上标记并保存

In [ ]:
%%time
@udf(returnType=StringType())
def add_flag(a):
    if a in ans:
        return '1'
    elif a in st_1:
        return '-1'
    else:
        return '0'
for i in tqdm(range(6, 10)):
    path = "../data_20210{}.tsv".format(i)
    df = spark.read.option("encoding","gb18030").csv(path,sep='\t',header=True)
    df =  df.filter(df['ITEM_SALES_VOLUME'] != 'null')
    df = df.na.fill('Unkown')
    df = df.withColumn('CATE_NAME_LV', F.concat(df['CATE_NAME_LV1'], df['CATE_NAME_LV2'], df['CATE_NAME_LV3'], df['CATE_NAME_LV4'],df['CATE_NAME_LV5']))
    df = df.withColumn("ITEM_ID", add_(df["ITEM_ID"]))
    df = df.withColumn('tmp', F.concat('ITEM_ID', 'DATA_MONTH'))
    df = df.withColumn("flag", add_flag(df["tmp"]))
    df.createOrReplaceTempView("data")
    if not os.path.exists('./H_{}'.format(i)):
        tmp = spark.sql('select * from data where flag = "1"')
        tmp.repartition(1).write.option("encoding","gb18030").csv('./H_{}'.format(i))

  0%|          | 0/4 [00:00<?, ?it/s]